In [2]:
import googlemaps
import pandas

In [3]:
df = pandas.read_csv("2014_ts.csv",index_col=0)

In [4]:
df.head()

,posTime,Lat,Long,altTime,Altitude,DistanceMeters,weekday
0,2014-10-16 07:48:11-07:00,37.3943,-122.0769,2014-10-16 07:48:11-07:00,19.3638,4780.8071,3
1,2014-10-16 07:48:12-07:00,37.3943,-122.0771,2014-10-16 07:48:12-07:00,19.3638,4791.6362,3
2,2014-10-16 07:48:17-07:00,37.3944,-122.0774,2014-10-16 07:48:17-07:00,19.8444,4826.5288,3
3,2014-10-16 07:48:26-07:00,37.3947,-122.0781,2014-10-16 07:48:26-07:00,19.8444,4895.5474,3
4,2014-10-16 07:48:29-07:00,37.3948,-122.0782,2014-10-16 07:48:29-07:00,19.8444,4916.4795,3


In [22]:
locations = [(row[1],row[2]) for row in df[['Lat','Long']].itertuples()]
locations[0]

(37.394300000000001, -122.07689999999999)

In [38]:
with open('api_key.txt', 'r') as api_key:
    key = api_key.read()
    gmaps = googlemaps.Client(key=key)
    test = googlemaps.geocoding.reverse_geocode(client=gmaps, latlng=locations[0])
    #addresses = [gmaps.geocoding.reverse_geocode(loc) for loc in locations]

HTTPError: HTTP Error: 404

In [30]:
dir(googlemaps)

['Client',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'client',
 'convert',
 'directions',
 'distance_matrix',
 'elevation',
 'exceptions',
 'geocoding',
 'googlemaps',
 'roads',
 'timezone']

In [36]:
print(googlemaps.geocoding.reverse_geocode.__doc__)


    Reverse geocoding is the process of converting geographic coordinates into a
    human-readable address.

    :param latlng: The latitude/longitude value for which you wish to obtain the
                   closest, human-readable address
    :type latlng: dict or list or tuple

    :param result_type: One or more address types to restrict results to.
    :type result_type: string or list of string

    :param location_type: One or more location types to restrict results to.
    :type location_type: list of string

    :param language: The language in which to return results.
    :type langauge: string

    :rtype: list of reverse geocoding results.

    
